In [1]:
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

NUM_CONT_FEATS = 10  # первые 10 колонок в feat_mat: cont_all

def compute_normalization_stats(train_sequences, num_cont_feats=NUM_CONT_FEATS):
    """
    train_sequences: список np.ndarray [T, F]
    Считаем mean/std только по первым num_cont_feats признакам на всём трейн-сете.
    """
    all_rows = []
    for seq in train_sequences:
        base = seq[:, :num_cont_feats]  # только continuous
        all_rows.append(base.reshape(-1, num_cont_feats))
    all_rows = np.concatenate(all_rows, axis=0)

    mean = all_rows.mean(axis=0)
    std = all_rows.std(axis=0)
    std[std < 1e-6] = 1.0  # чтобы не делить на 0
    return mean, std


def apply_normalization(sequences, mean, std, num_cont_feats=NUM_CONT_FEATS):
    """
    Применяем (x - mean) / std к первым num_cont_feats фичам.
    Остальное (бинарные и one-hot) не трогаем.
    """
    normed = []
    for seq in sequences:
        seq = seq.copy()
        seq[:, :num_cont_feats] = (seq[:, :num_cont_feats] - mean) / std
        normed.append(seq)
    return normed

def build_encoders(pbp: pd.DataFrame) -> Dict[str, Dict[str, int]]:
    """
    Строим один словарь для комбинированного типа события:
    combined_type = EVENT_TYPE  или  EVENT_TYPE_SECONDARYTYPE
    (secondary_type с NaN не даёт отдельной категории)
    """
    encoders: Dict[str, Dict[str, int]] = {}

    et = (
        pbp["event_type"]
        .astype(str)
        .str.upper()
    )

    st = pbp["secondary_type"].astype("string")

    combined = []
    for e, s in zip(et, st):
        if pd.isna(s) or s == "":
            combined.append(e)                       # только event_type
        else:
            combined.append(f"{e}_{str(s).upper()}") # event_type_secondary_type

    combined_unique = sorted(set(combined))
    encoders["event_type_combined"] = {ct: i for i, ct in enumerate(combined_unique)}

    return encoders


########################################
# 2. Построение выборок по минутам периода
########################################

def build_game_sequences(
    pbp: pd.DataFrame,
    encoders: Dict[str, Dict[str, int]],
    min_events_per_game: int = 50,
    window_size: int = 50,
    max_period: int = 3,  # берём только 1–3 периоды
) -> Tuple[List[np.ndarray], List[int], List[dict]]:
    """
    Делаем выборки для обучения:

    Каждая выборка = 50 последних событий перед минутной отметкой в периоде.
    Лейбл = результат периода: 1, если home ведёт после конца периода, иначе 0.

    Возвращает:
      - X_list: список np.ndarray[window_size, F]
      - y_list: список int (0/1)
      - meta_list: список словарей с метой:
            { "game_id": ..., "period": ..., "minute_in_period": ..., "evant_time": ... }
    """

    required_cols = [
        "game_id", "event_type", "period", "game_seconds",
        "home_score", "away_score",
        "event_team_type",
        "x_fixed", "y_fixed",
        "shot_distance", "shot_angle",
        "home_skaters", "away_skaters",
        "extra_attacker",
        "secondary_type",
    ]
    missing = [c for c in required_cols if c not in pbp.columns]
    if missing:
        raise ValueError(f"Missing required columns in pbp: {missing}")

    # сортируем внутри игры по времени
    pbp_sorted = pbp.sort_values(["game_id", "game_seconds"]).reset_index(drop=True)

    X_list: List[np.ndarray] = []
    y_list: List[int] = []
    meta_list: List[dict] = []

    event_type_to_idx = encoders["event_type_combined"]

    for game_id, g in pbp_sorted.groupby("game_id", sort=False):
        if len(g) < max(min_events_per_game, window_size):
            continue

        # сброс индекса внутри игры
        g = g.reset_index(drop=True)
        N = len(g)

        # общие массивы по игре
        game_seconds = g["game_seconds"].astype(float).values
        max_t = max(game_seconds.max(), 1.0)
        t_norm = game_seconds / max_t

        period_arr = g["period"].astype(int).values

        home_score_all = g["home_score"].astype(float).values
        away_score_all = g["away_score"].astype(float).values
        score_diff_all = home_score_all - away_score_all

        home_skaters_all = g["home_skaters"].astype(float).fillna(0.0).values
        away_skaters_all = g["away_skaters"].astype(float).fillna(0.0).values

        extra_attacker_all = g["extra_attacker"].astype(float).fillna(0.0).values

        is_home_event_all = g["event_team_type"].eq("home").fillna(False).astype(float).values
        is_away_event_all = g["event_team_type"].eq("away").fillna(False).astype(float).values

        x_fixed_all = g["x_fixed"].astype(float).fillna(0.0).values
        y_fixed_all = g["y_fixed"].astype(float).fillna(0.0).values
        shot_distance_all = g["shot_distance"].astype(float).fillna(0.0).values
        shot_angle_all = g["shot_angle"].astype(float).fillna(0.0).values

        cont_all = np.stack(
            [
                t_norm,
                home_score_all,
                away_score_all,
                score_diff_all,
                home_skaters_all,
                away_skaters_all,
                x_fixed_all,
                y_fixed_all,
                shot_distance_all,
                shot_angle_all,
            ],
            axis=1,
        ).astype(np.float32)

        # бинарные фичи [N, 3]
        binary_all = np.stack(
            [
                extra_attacker_all,
                is_home_event_all,
                is_away_event_all,
            ],
            axis=1,
        ).astype(np.float32)

        # period -> one-hot [N, max_period]
        num_periods = max_period
        period_ohe_all = np.zeros((N, num_periods), dtype=np.float32)
        for i, p in enumerate(period_arr):
            if 1 <= p <= num_periods:
                period_ohe_all[i, p - 1] = 1.0
        
        # комбинированный тип события по всей игре
        et = g["event_type"].astype(str).str.upper().values
        st = g["secondary_type"].astype("string").values

        combined = []
        for e, s in zip(et, st):
            if pd.isna(s) or s == "":
                combined.append(e)
            else:
                combined.append(f"{e}_{str(s).upper()}")

        num_types = len(event_type_to_idx)
        type_ohe_all = np.zeros((N, num_types), dtype=np.float32)
        for i, ct in enumerate(combined):
            j = event_type_to_idx.get(ct)
            if j is not None:
                type_ohe_all[i, j] = 1.0

        # === по периодам ===
        for p in range(1, max_period + 1):
            period_mask = (period_arr == p)
            if not period_mask.any():
                continue

            idxs_p = np.where(period_mask)[0]
            times_p = game_seconds[idxs_p]

            # --- считаем ГОЛЫ ИМЕННО В ЭТОМ ПЕРИОДЕ ---
            first_idx = idxs_p[0]
            last_idx  = idxs_p[-1]

            # счёт перед стартом периода
            if first_idx > 0:
                home_before = home_score_all[first_idx - 1]
                away_before = away_score_all[first_idx - 1]
            else:
                # для 1-го периода до начала игры счёт 0:0
                home_before = 0.0
                away_before = 0.0

            # счёт после конца периода (накопительный)
            home_end = home_score_all[last_idx]
            away_end = away_score_all[last_idx]

            # голы, забитые ТОЛЬКО в этом периоде
            home_goals_period = home_end - home_before
            away_goals_period = away_end - away_before

            # таргет: кто выиграл период
            if home_goals_period > away_goals_period:
                home_win_period = 1
            else:
                # и ничьи, и победу гостей кодируем как 0
                home_win_period = 0

            # 20 минут в периоде → минутные границы
            period_start = (p - 1) * 20 * 60  # 0, 1200, 2400
            # minute_in_period = 1..20
            for minute_in_period in range(1, 21):
                threshold = period_start + 60 * minute_in_period

                # ищем первое событие этого периода, у которого game_seconds >= threshold
                pos_in_p = np.searchsorted(times_p, threshold, side="left")
                if pos_in_p >= len(times_p):
                    # в этом периоде после этой минуты уже не было событий
                    continue

                idx_event = idxs_p[pos_in_p]  # индекс события в g (0..N-1)
                if idx_event < window_size - 1:
                    # недостаточно 50 предыдущих событий
                    continue

                # окно из 50 событий (включая событие на минутной границе)
                start_idx = idx_event - window_size + 1
                end_idx = idx_event + 1  # срез до end_idx не включительно
                window_idx = np.arange(start_idx, end_idx)

                cont = cont_all[window_idx]            # [T, 10]
                binary = binary_all[window_idx]        # [T, 3]
                period_ohe = period_ohe_all[window_idx]# [T, 3]
                type_ohe = type_ohe_all[window_idx]    # [T, num_types]

                feat_mat = np.concatenate(
                    [cont, binary, period_ohe, type_ohe],
                    axis=1
                )  # [T, 10+3+3+num_types]

                X_list.append(feat_mat)
                y_list.append(home_win_period)
                meta_list.append(
                    {
                        "game_id": game_id,
                        "period": int(p),
                        "minute_in_period": int(minute_in_period),
                        "event_time": float(game_seconds[idx_event]),
                    }
                )

    y_arr = np.array(y_list, dtype=float)
    print("y NaN:", np.isnan(y_arr).sum())
    print("y уникальные значения:", np.unique(y_arr))

    return X_list, y_list, meta_list


########################################
# 3. Dataset + collate_fn для PyTorch
########################################

class HockeySeqDataset(Dataset):
    """
    Простой Dataset: один объект = одна выборка (50 событий).
    x: [T, F], y: 0/1 (home ведёт после периода), meta: словарь с метой.
    """
    def __init__(self, sequences: List[np.ndarray], labels: List[int], metas: List[dict]):
        assert len(sequences) == len(labels) == len(metas)
        self.sequences = sequences
        self.labels = labels
        self.metas = metas

    def __len__(self) -> int:
        return len(self.sequences)

    def __getitem__(self, idx: int):
        x = torch.from_numpy(self.sequences[idx])  # [T, F]
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        meta = self.metas[idx]
        return x, y, meta


def collate_pad(batch):
    """
    Пакуем батч в общий тензор [B, T_max, F] + маска [B, T_max]
    и прокидываем meta как список словарей.
    """
    xs, ys, metas = zip(*batch)  # <--- добавили metas
    lengths = [x.size(0) for x in xs]
    F = xs[0].size(1)
    B = len(xs)
    T_max = max(lengths)

    padded = xs[0].new_zeros((B, T_max, F))
    mask = torch.zeros((B, T_max), dtype=torch.bool)  # True на реальных шагах

    for i, x in enumerate(xs):
        L = x.size(0)
        padded[i, :L] = x
        mask[i, :L] = True

    y = torch.stack(ys)
    metas = list(metas)  # просто список словарей
    return padded, y, mask, metas


In [2]:
########################################
# 4. Модели
########################################

class LSTMOutcomeModel(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int = 128, num_layers: int = 2, dropout: float = 0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=False
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        # x: [B, T, F], mask: [B, T], True на реальных шагах
        out, (h_n, c_n) = self.lstm(x)  # out: [B, T, H]
        # берём скрытое состояние последнего шага по маске
        last_indices = (mask.sum(dim=1) - 1).clamp(min=0)
        batch_idx = torch.arange(x.size(0), device=x.device)
        last_hidden = out[batch_idx, last_indices]  # [B, H]
        logits = self.fc(self.dropout(last_hidden)).squeeze(-1)
        return logits  # без сигмоиды, подадим в BCEWithLogitsLoss


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model, dtype=torch.float32)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float32) * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.cos(position * div_term[:-1])
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: [B, T, d_model]
        T = x.size(1)
        x = x + self.pe[:, :T]
        return x


class TransformerOutcomeModel(nn.Module):
    def __init__(
        self,
        input_dim: int,
        d_model: int = 128,
        nhead: int = 4,
        num_layers: int = 3,
        dim_feedforward: int = 256,
        dropout: float = 0.2
    ):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        """
        x: [B, T, F]
        mask: [B, T] (True на реальных шагах) -> нужно инвертировать для src_key_padding_mask
        """
        x = self.input_proj(x)
        x = self.pos_encoder(x)
        src_key_padding_mask = ~mask  # [B, T]
        enc_out = self.encoder(x, src_key_padding_mask=src_key_padding_mask)  # [B, T, d_model]
        last_indices = (mask.sum(dim=1) - 1).clamp(min=0)
        batch_idx = torch.arange(x.size(0), device=x.device)
        last_hidden = enc_out[batch_idx, last_indices]  # [B, d_model]
        logits = self.fc(self.dropout(last_hidden)).squeeze(-1)
        return logits  # логиты для BCEWithLogitsLoss


########################################
# 5. Обучение и валидация
########################################

def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    device: torch.device
) -> float:
    model.train()
    total_loss = 0.0
    criterion = nn.BCEWithLogitsLoss()
    for x, y, mask, metas in loader: 
        x = x.to(device)
        y = y.float().to(device)
        mask = mask.to(device)
        optimizer.zero_grad()
        logits = model(x, mask)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)


@torch.no_grad()
def evaluate(
    model: nn.Module,
    loader: DataLoader,
    device: torch.device
) -> Tuple[float, float]:
    """
    Возвращает (loss, accuracy) по всей валидации.
    """
    model.eval()
    criterion = nn.BCEWithLogitsLoss()
    total_loss = 0.0
    correct = 0
    total = 0
    for x, y, mask, metas in loader:  # metas игнорируем здесь
        x = x.to(device)
        y = y.float().to(device)
        mask = mask.to(device)
        logits = model(x, mask)
        loss = criterion(logits, y)
        total_loss += loss.item() * x.size(0)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).long()
        correct += (preds.cpu() == y.long().cpu()).sum().item()
        total += x.size(0)
    return total_loss / len(loader.dataset), correct / max(total, 1)

@torch.no_grad()
def evaluate_by_minute_and_period(
    model: nn.Module,
    loader: DataLoader,
    device: torch.device
) -> None:
    """
    Считает accuracy на валидации отдельно для каждой (period, minute_in_period).
    Печатает табличку вида:
      Period 1, minute  1: acc=0.63 (n=123)
    """
    model.eval()
    bucket_stats = {}  # (period, minute) -> {"correct": int, "total": int}

    for x, y, mask, metas in loader:
        x = x.to(device)
        y = y.float().to(device)
        mask = mask.to(device)

        logits = model(x, mask)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).long().cpu()
        y_true = y.long().cpu()

        correct_batch = (preds == y_true).numpy()

        # идём по объектам в батче
        for is_corr, meta in zip(correct_batch, metas):
            period = int(meta["period"])
            minute = int(meta["minute_in_period"])
            key = (period, minute)
            if key not in bucket_stats:
                bucket_stats[key] = {"correct": 0, "total": 0}
            bucket_stats[key]["correct"] += int(is_corr)
            bucket_stats[key]["total"] += 1

    # печатаем аккуратно по периодам и минутам
    periods = sorted({p for (p, m) in bucket_stats.keys()})
    for p in periods:
        print(f"\n=== Period {p} ===")
        minutes = sorted(m for (pp, m) in bucket_stats.keys() if pp == p)
        for m in minutes:
            stats = bucket_stats[(p, m)]
            acc = stats["correct"] / stats["total"]
            print(f"Minute {m:2d}: acc={acc:.3f} (n={stats['total']})")


In [3]:
# === Конфиг ===
batch_size = 64
num_epochs = 20
lr = 1e-3
use_transformer = True  # False -> LSTM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# === Загрузка данных ===
df1 = pd.read_csv("/kaggle/input/data-21-24/play_by_play_2021_22.csv", encoding="latin1")
df2 = pd.read_csv("/kaggle/input/data-21-24/play_by_play_2022_23.csv", encoding="latin1")
df3 = pd.read_csv("/kaggle/input/data-21-24/play_by_play_2023_24.csv", encoding="latin1")
df = pd.concat([df1, df2, df3], ignore_index=True)

# убираем смены
df_model = df[df["event_type"] != "CHANGE"].copy()

# === Энкодеры и выборки ===
encoders = build_encoders(df_model)
X_list, y_list, meta_list = build_game_sequences(
    df_model,
    encoders,
    min_events_per_game=50,
    window_size=50,
    max_period=3,
)

# meta_list содержит game_id
all_game_ids = [m["game_id"] for m in meta_list]
unique_game_ids = sorted(set(all_game_ids))

# например, 80% матчей в train, 20% в val
rng = np.random.default_rng(seed=42)
rng.shuffle(unique_game_ids)

n_games = len(unique_game_ids)
n_train_games = int(0.8 * n_games)
train_game_ids = set(unique_game_ids[:n_train_games])
val_game_ids   = set(unique_game_ids[n_train_games:])

# индексы выборок по матчам
train_idx = [i for i, m in enumerate(meta_list) if m["game_id"] in train_game_ids]
val_idx   = [i for i, m in enumerate(meta_list) if m["game_id"] in val_game_ids]

X_train_raw = [X_list[i] for i in train_idx]
y_train     = [y_list[i] for i in train_idx]
meta_train  = [meta_list[i] for i in train_idx]

X_val_raw = [X_list[i] for i in val_idx]
y_val     = [y_list[i] for i in val_idx]
meta_val  = [meta_list[i] for i in val_idx]

# --- считаем нормировку по train ---
mean, std = compute_normalization_stats(X_train_raw)

# --- применяем к train и val ---
X_train = apply_normalization(X_train_raw, mean, std)
X_val   = apply_normalization(X_val_raw,   mean, std)

# строим датасеты уже из нормализованных фич
train_ds = HockeySeqDataset(X_train, y_train, meta_train)
val_ds   = HockeySeqDataset(X_val,   y_val,   meta_val)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  collate_fn=collate_pad)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, collate_fn=collate_pad)

# === Модель ===
input_dim = train_ds[0][0].shape[1]
if use_transformer:
    model = TransformerOutcomeModel(input_dim=input_dim, d_model=128,
                                    nhead=4, num_layers=2,  # можно уменьшить слои
                                    dim_feedforward=256,
                                    dropout=0.3)
else:
    model = LSTMOutcomeModel(input_dim=input_dim, hidden_dim=128,
                             num_layers=2, dropout=0.3)

model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',   # хотим минимизировать val_loss
    factor=0.5,   # уменьшать lr в 2 раза
    patience=2,   # если 2 эпохи нет улучшения — режем lr
    verbose=True
)

# === Тренировка ===
for epoch in range(1, num_epochs + 1):
    train_loss = train_one_epoch(model, train_loader, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, device)
    print(f"Epoch {epoch}: "
          f"train_loss={train_loss:.4f} "
          f"val_loss={val_loss:.4f} "
          f"val_acc={val_acc:.4f}")

    scheduler.step(val_loss)

Device: cuda


/tmp/ipykernel_47/3668448661.py:11: DtypeWarning: Columns (49,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("/kaggle/input/data-21-24/play_by_play_2021_22.csv", encoding="latin1")
/tmp/ipykernel_47/3668448661.py:12: DtypeWarning: Columns (48,49,56) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/kaggle/input/data-21-24/play_by_play_2022_23.csv", encoding="latin1")
/tmp/ipykernel_47/3668448661.py:13: DtypeWarning: Columns (42,43,49,50,120,136) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("/kaggle/input/data-21-24/play_by_play_2023_24.csv", encoding="latin1")


y NaN: 0
y уникальные значения: [0. 1.]


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(


Epoch 1: train_loss=0.5051 val_loss=0.5039 val_acc=0.7696
Epoch 2: train_loss=0.4744 val_loss=0.4683 val_acc=0.7822
Epoch 3: train_loss=0.4616 val_loss=0.4878 val_acc=0.7635
Epoch 4: train_loss=0.4525 val_loss=0.4690 val_acc=0.7756
Epoch 5: train_loss=0.4458 val_loss=0.4660 val_acc=0.7824
Epoch 6: train_loss=0.4438 val_loss=0.4649 val_acc=0.7807
Epoch 7: train_loss=0.4406 val_loss=0.4507 val_acc=0.7857
Epoch 8: train_loss=0.4386 val_loss=0.4646 val_acc=0.7789
Epoch 9: train_loss=0.4372 val_loss=0.4686 val_acc=0.7853
Epoch 10: train_loss=0.4358 val_loss=0.4514 val_acc=0.7823
Epoch 11: train_loss=0.4287 val_loss=0.4540 val_acc=0.7852
Epoch 12: train_loss=0.4241 val_loss=0.4520 val_acc=0.7864
Epoch 13: train_loss=0.4219 val_loss=0.4631 val_acc=0.7837
Epoch 14: train_loss=0.4166 val_loss=0.4517 val_acc=0.7904
Epoch 15: train_loss=0.4142 val_loss=0.4660 val_acc=0.7861
Epoch 16: train_loss=0.4129 val_loss=0.4537 val_acc=0.7896
Epoch 17: train_loss=0.4103 val_loss=0.4520 val_acc=0.7907
Epoch 

In [4]:
evaluate_by_minute_and_period(model, val_loader, device)


=== Period 1 ===
Minute  5: acc=0.500 (n=2)
Minute  6: acc=0.789 (n=19)
Minute  7: acc=0.780 (n=82)
Minute  8: acc=0.811 (n=212)
Minute  9: acc=0.837 (n=375)
Minute 10: acc=0.826 (n=564)
Minute 11: acc=0.829 (n=692)
Minute 12: acc=0.842 (n=766)
Minute 13: acc=0.835 (n=812)
Minute 14: acc=0.862 (n=825)
Minute 15: acc=0.874 (n=832)
Minute 16: acc=0.891 (n=834)
Minute 17: acc=0.917 (n=836)
Minute 18: acc=0.946 (n=837)
Minute 19: acc=0.973 (n=837)
Minute 20: acc=0.999 (n=837)

=== Period 2 ===
Minute  1: acc=0.644 (n=837)
Minute  2: acc=0.667 (n=837)
Minute  3: acc=0.686 (n=837)
Minute  4: acc=0.686 (n=837)
Minute  5: acc=0.704 (n=837)
Minute  6: acc=0.728 (n=837)
Minute  7: acc=0.740 (n=837)
Minute  8: acc=0.768 (n=837)
Minute  9: acc=0.791 (n=837)
Minute 10: acc=0.802 (n=837)
Minute 11: acc=0.812 (n=837)
Minute 12: acc=0.803 (n=837)
Minute 13: acc=0.803 (n=837)
Minute 14: acc=0.804 (n=837)
Minute 15: acc=0.814 (n=837)
Minute 16: acc=0.808 (n=837)
Minute 17: acc=0.804 (n=837)
Minute 18: 